<a href="https://colab.research.google.com/github/matinmazid/York_ML_certificate/blob/master/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load all the dependant libraries

# Scenario 0 No context provided to model

In [1]:
!pip --quiet install langchain langchain_community pypdf langchain_core langchain-text-splitters langchain-chroma
!pip --quiet install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 41.3 MB/s eta 0:00:

In [8]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPEN_API')

Create the lang chain

In [9]:
from langchain.chat_models import ChatOpenAI
import langchain_core
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
model = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()
chatPrompt=ChatPromptTemplate.from_messages(
    [("system","you are a financial specialist with experience in Aritifical Intelligence"),
     ("user","what is XBRL and its relationship to AICPA")]
    )

In [10]:
chain= chatPrompt | model | output_parser

In [11]:
chain.invoke({})


'XBRL stands for eXtensible Business Reporting Language, which is a standardized format for financial reporting. It allows companies to tag financial data in their reports so that it can be easily analyzed and compared across different organizations. XBRL helps to streamline the process of financial reporting and makes it more efficient and accurate.\n\nThe American Institute of Certified Public Accountants (AICPA) is a professional organization for accountants in the United States. The AICPA has been involved in promoting the use of XBRL for financial reporting. They have developed guidelines and best practices for using XBRL in financial statements to ensure consistency and accuracy in reporting.\n\nIn summary, XBRL is a standardized format for financial reporting, and the AICPA has been actively involved in promoting its use and providing guidance on how to implement it effectively in financial reporting processes.'

# Scenario 1 Load pdf document loader saving to Chromadb

In [6]:
!pip install --upgrade --quiet pypdf
!pip install --upgrade --quiet langchain_openai
!pip install --quiet tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.6 MB/s eta 0:00:00


https://www.stern.nyu.edu/sites/default/files/assets/documents/ChenChoDouLev2021WP.pdf

https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

In [12]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import tiktoken
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

loader = PyPDFLoader("ChenChoDouLev2021WP.pdf")
docs = loader.load()


# split and chuck the input data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs)

# all_texts = [cleanStr(doc.page_content) for doc in all_splits]
# all_texts


vectorstore = Chroma.from_documents(documents=all_splits,
                                    embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("what is XBRL and its relationship to AICPA")




/usr/local/lib/python3.11/dist-packages/langsmith/client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"XBRL is the eXtensible Business Reporting Language used for financial reporting. The AICPA is involved in developing data quality tools and working with the FASB on U.S. GAAP taxonomy enhancements related to XBRL. Plumlee and Plumlee (2008) and Boritz and No (2009) discuss the potential challenges of XBRL documents' assurance."

# Scenario 2 Pine cone database and webcrawler
https://python.langchain.com/docs/integrations/document_loaders/recursive_url/
https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm


In [13]:
!pip install -qU langchain-community beautifulsoup4 lxml

In [14]:
import re
from bs4 import BeautifulSoup
# import html as htmlTool


def bs4_extractor(html: str) -> str:
    soup = BeautifulSoup(html, "lxml")
    someText=re.sub(r"\n\n+", "\n\n", soup.text).strip()
    # someText=htmlTool.unescape(someText)
    # the test page had non-breaking space as well as other non acii characters
    # we get rid of them
    return someText.encode('latin-1', 'ignore').decode('ascii',"ignore")


In [15]:
from langchain_community.document_loaders import RecursiveUrlLoader

loader=RecursiveUrlLoader(url="https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm",
                           use_async=False
                          ,extractor=bs4_extractor)

docs_html = loader.load()

# text splitter knows how to extract the text of a sourced doc
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
# text splitter knows how to extract the text of a sourced doc
all_splits = text_splitter.split_documents(docs_html)


In [16]:
all_splits[3]

Document(metadata={'source': 'https://www150.statcan.gc.ca/n1/daily-quotidien/250207/dq250207a-eng.htm', 'content_type': 'text/html; charset=UTF-8', 'title': 'The Daily\xa0—\xa0Labour Force Survey, January 2025', 'description': 'Employment increased by 76,000 (+0.4%) in January and the employment rate rose 0.1 percentage points to 61.1%. The unemployment rate declined 0.1 percentage points to 6.6%.', 'language': 'en'}, page_content='527,000\nJanuary2025\n\n-0.1%\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\n\r\nUnemployment rate  N.S.\r\n\n5.9%\nJanuary2025\n\n-0.4pts\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\nTab1\nTab2\n\n\r\nEmployment  N.B.\r\n\n403,000\nJanuary2025\n\n0.7%\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\n\r\nUnemployment rate  N.B.\r\n\n6.4%\nJanuary2025\n\n-1.3pts\n(monthly change)\n\n Source(s): Table 14-10-0287-01.\r\n\nTab1\nTab2\n\n\r\nEmployment  Que.\r\n\n4,636,000\nJanuary2025\n\n0.3%\n(monthly change)')

Pinecone requires a API key.
https://docs.pinecone.io/guides/get-started/quickstart

In [17]:
!pip install --quiet protobuf==3.20.*
!pip install  --quiet "pinecone[grpc]"
!pip install --upgrade --quiet  lark
!pip install --upgrade --quiet pinecone-notebooks pinecone-client
!pip install --upgrade --quiet langchain_pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.30.0 requires protobuf<6.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 18.1 MB/s eta 0:00:00

In [19]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

pc = Pinecone(api_key=userdata.get('PINECONE'))
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE')

In [20]:
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

index_name = "york-1020-assignment-2"
# create new index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


pinecone_vector_store= PineconeVectorStore.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(),
    index_name=index_name,
    text_key="page_content"
)

pinecone_retriever = pinecone_vector_store.as_retriever()
#
pinecone_rag_chain = (
    {"context": pinecone_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
pinecone_rag_chain.invoke("how much has employment increased")

'Employment increased by 76,000 (+0.4%) in January, with the employment rate rising by 0.1 percentage points to 61.1%. The recent increases in employment follow a period where growth had been outpaced by population growth.'

# Scenario 3 Dropbox as cloud storage and weaviate

In [41]:
!pip install --quiet dropbox
!pip install --quiet "unstructured[pdf]"
!pip install -U --quiet weaviate-client
!pip install --qU unstructured

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --qU


https://python.langchain.com/docs/integrations/document_loaders/dropbox/
https://python.langchain.com/docs/integrations/vectorstores/weaviate/#connecting-to-weaviate
https://weaviate.io/developers/weaviate/quickstart/local


In [42]:
from langchain_community.document_loaders import DropboxLoader
# Generate access token: https://www.dropbox.com/developers/apps/create.
dropbox_access_token = userdata.get('DROP_BOX')
# Dropbox root folder
dropbox_folder_path = ""
loader = DropboxLoader(
    dropbox_access_token=dropbox_access_token,
    dropbox_folder_path=dropbox_folder_path,
    recursive=True,
)

In [43]:
documents = loader.load()


File /trust-worthy-artificial-intelligence.pdf type detected as .pdf
File /ai-future-trends.pdf type detected as .pdf


Load weaveate
https://weaviate.io/developers/weaviate/client-libraries/python

In [ ]:
!pip install -U weaviate-client
!pip install -Uq langchain-weaviate
!pip install openai tiktoken langchain

In [66]:
import weaviate
import os

# Best practice: store your credentials in environment variables
openai_api_key = os.environ["OPENAI_API_KEY"]
# this is not rerunable. Check error
# this line seems to start the weaveate server
weaviate_client = weaviate.connect_to_embedded(
      version="1.26.1",
      headers={
          "X-OpenAI-Api-Key": openai_api_key
      },
  )





WeaviateStartUpError: Embedded DB did not start because processes are already listening on ports http:8079 and grpc:50050use weaviate.connect_to_local(port=8079, grpc_port=50050) to connect to the existing instance

In [62]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [78]:
documents[1]

Document(metadata={'source': '/tmp/tmpl_w82suo/tmp.pdf'}, page_content='The current issue and full text archive of this journal is available on Emerald Insight at: https://www.emerald.com/insight/0959-6119.htm\n\nArtiﬁcial intelligence: an overview of research trends and future directions Dogan Gursoy School of Hospitality Business Management, Washington State University, Pullman, Washington, USA and School of Tourism and Hospitality, University of Johannesburg, Johannesburg, South Africa, and\n\nRuiying Cai School of Hospitality Business Management, Washington State University, Pullman, Washington, USA\n\nAbstract Purpose – This study aims to offer an overview of hospitality and tourism research on artiﬁcial intelligence (AI) anditsimpact on theindustry. More speciﬁcally, this studyexamines hospitality andtourism AIresearch trends in hospitality and tourism customer service experience creation and delivery, service failure and recovery, human resources and organizational behavior. Bas

In [72]:
import weaviate
from langchain_weaviate.vectorstores import WeaviateVectorStore
weaviate_client_local = weaviate.connect_to_local( host="localhost", port=8079,
                                                  grpc_port=50050,
    headers={
          "X-OpenAI-Api-Key": openai_api_key
      },
)
embeddings = OpenAIEmbeddings()
db = WeaviateVectorStore.from_documents(docs, embeddings, client=weaviate_client_local)

In [ ]:
format_docs = lambda x: "\n\n".join([doc.page_content for doc in x])

In [74]:
weaviate_retriever = db.as_retriever()
#
weaviate_rag_chain = (
    {"context": weaviate_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [76]:
weaviate_rag_chain.invoke("What are the different categories of AI")

'The different categories of AI are artificial narrow AI, general AI, and super AI. Narrow AI is commonly applied in industries like hospitality and tourism for specific tasks. General AI is currently in development and aims to perform tasks intelligently similar to human capabilities.'